<a href="https://colab.research.google.com/github/mahomudgamalfcis/Image-Paragraph-Captioning/blob/master/DemoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1- Change Version of CUDA from 10.0 to 9.2


In [0]:
# Check for Current version
!nvcc --version
#--------------------------------------------------

# Uninstall the current version of CUDA
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
#---------------------------------------------------

# Install the new version of CUDA
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9-2
#---------------------------------------------------

# check if CUDA 9.2 is installed
!nvcc --version


#2- Setup Torch and It's Packages


In [0]:
# This Commands will install Torch with luaJIT
!git clone https://github.com/nagadomi/distro.git ~/torch --recursive
%cd ~/torch
!bash install-deps
!./update.sh
!./install.sh
#--------------------------------------------------

# The commands below will install set of packages to make the model run successfully
!/root/torch/install/bin/luarocks install torch
!/root/torch/install/bin/luarocks install cutorch
!/root/torch/install/bin/luarocks install cunn
!/root/torch/install/bin/luarocks  install https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/torch-rnn-scm-1.rockspec
''' this command is to install stnbhw-scm-1, but it is a modified repo. we modified the CMakeLists file to change GPU architecture to 
fit the one which installed in Googel Colab. the line we changed is 
 IF (CUDA_FOUND)
   LIST(APPEND CUDA_NVCC_FLAGS "-arch=sm_20") To 

 IF (CUDA_FOUND)
   LIST(APPEND CUDA_NVCC_FLAGS "-arch=sm_30")'''
!/root/torch/install/bin/luarocks install https://raw.githubusercontent.com/mahomudgamalfcis/stnbhwd/master/stnbhwd-scm-1.rockspec

# the commands below will install HDF5-1.10 
%cd /content
!git clone https://github.com/anibali/torch-hdf5.git
!cd torch-hdf5
!git checkout hdf5-1.10 
!/root/torch/install/bin/luarocks make hdf5-0-0.rockspec

# this command to display Installed packages by Luarocks
!/root/torch/install/bin/luarocks list

#3- Clone im2p repository 

In [0]:
!git clone https://github.com/mahomudgamalfcis/im2p.git

###- Download Visual Genome 
      we have two files to download 
      1- genome_VG_100K_path (9 GB)
      2- enome_VG_100K_path2 (5 GB)

In [0]:
# first download genome_VG_100K_path (9 GB)
!mkdir -p /content/im2p/data/VG
%cd /content/im2p/data/VG
!wget https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip
!unzip images.zip
!rm images.zip
%cd ../../../
#----------------------------------------------------------

# second download genome_VG_100K_path2 (5 GB)
!mkdir -p /content/im2p/data/VG2
%cd /content/im2p/data/VG2
!wget https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip
!unzip images2.zip
!rm images2.zip
%cd ../../../

### i- Run split_dataset.py file 
    this file is responsible for split our dataset to three parts
      1- train dataset
      2- test dataset
      3- validation dataset
    then it create a text file for each splitted dataset which contains all image paths to use it in extract feature step 


In [0]:
!python /content/im2p/split_dataset.py

###ii- Run get_imgs_train_path.py file
    this file is responsible for getting all paths of images in training dataset and save it in another text file 

In [0]:
!python /content/im2p/get_imgs_train_path.py

###iii- Run get_imgs_test_path.py file
  this file is responsible for getting all paths of images in testing dataset and save it in another text file

In [0]:
!python /content/im2p/get_imgs_test_path.py

###iv- Run get_imgs_val_path.py file
    this file is responsible for getting all paths of images in validation dataset and save it in another text file 

In [0]:
!python /content/im2p/get_imgs_val_path.py

###v- Download pre-trained model of Encoder

In [0]:
!mkdir -p data/models/densecap
%cd ./data/models/densecap
!wget http://cs.stanford.edu/people/jcjohns/densecap/densecap-pretrained-vgg16.t7.zip
!unzip densecap-pretrained-vgg16.t7.zip
!rm densecap-pretrained-vgg16.t7.zip
%cd ../../../

###vi- Extract feature for Training dataset

In [0]:
!/root/torch/install/bin/th /content/im2p/extract_features.lua -boxes_per_image 50 -max_images -1 -checkpoint /content/im2p/data/models/densecap/densecap-pretrained-vgg16.t7  -input_txt /content/im2p/imgs_train_path.txt \
                          -output_h5 ./data/im2p_train_output.h5 -gpu 0 -use_cudnn 1

###vii- Extract feature for Testing dataset

In [0]:
!/root/torch/install/bin/th /content/im2p/extract_features.lua -boxes_per_image 50 -max_images -1 -checkpoint /content/im2p/data/models/densecap/densecap-pretrained-vgg16.t7  -input_txt /content/im2p/imgs_test_path.txt \
                          -output_h5 ./data/im2p_test_output.h5 -gpu 0 -use_cudnn 1

###viii- Extract feature for Validation dataset

In [0]:
!/root/torch/install/bin/th /content/im2p/extract_features.lua -boxes_per_image 50 -max_images -1 -checkpoint /content/im2p/data/models/densecap/densecap-pretrained-vgg16.t7  -input_txt /content/im2p/imgs_val_path.txt \
                          -output_h5 ./data/im2p_val_output.h5 -gpu 0 -use_cudnn 1

#4- Check the Output of Encoder

In [0]:
import h5py
f = h5py.File(r'/content/im2p/data/im2p_train_output.h5')
list(f)
f['feats'].shape